In [1]:
import os
import time
import logging
import argparse

from utils.hparams import HParam
from utils.writer import MyWriter
from datasets.dataloader import create_dataloader

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('-b', '--base_dir', type=str, default='.',
                    help="Root directory of run.")
parser.add_argument('-c', '--config', type=str, required=True,
                    help="yaml file for configuration")
parser.add_argument('-e', '--embedder_path', type=str, required=True,
                    help="path of embedder model pt file")
parser.add_argument('--checkpoint_path', type=str, default=None,
                    help="path of checkpoint pt file")
parser.add_argument('-m', '--model', type=str, required=True,
                    help="Name of the model. Used for both logging and saving checkpoints.")
args = parser.parse_args(["-c", "config.yaml", "-e", "embedder.pt", "-m", "first_try"])
hp = HParam(args.config)

In [3]:
with open(args.config, 'r') as f:
    # store hparams as string
    hp_str = ''.join(f.readlines())

pt_dir = os.path.join(args.base_dir, hp.log.chkpt_dir, args.model)
os.makedirs(pt_dir, exist_ok=True)

log_dir = os.path.join(args.base_dir, hp.log.log_dir, args.model)
os.makedirs(log_dir, exist_ok=True)

chkpt_path = args.checkpoint_path if args.checkpoint_path is not None else None

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir,
            '%s-%d.log' % (args.model, time.time()))),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

if hp.data.train_dir == '' or hp.data.test_dir == '':
    logger.error("train_dir, test_dir cannot be empty.")
    raise Exception("Please specify directories of data in %s" % args.config)

writer = MyWriter(hp, log_dir)

trainloader = create_dataloader(hp, args, train=True)
testloader = create_dataloader(hp, args, train=False)

In [4]:
import os
import math
import torch
import torch.nn as nn
import traceback

from utils.adabound import AdaBound
from utils.audio import Audio
from utils.evaluation import validate
from model.model import VoiceFilter
from model.embedder import SpeechEmbedder
from utils.power_law_loss import PowerLawCompLoss
from utils.gdrive import GDrive

In [5]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [6]:
# load embedder
embedder_pt = torch.load(args.embedder_path)
embedder = SpeechEmbedder(hp).cuda()
embedder.load_state_dict(embedder_pt)
embedder.eval()

print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   95746 KB |  141609 KB |  142378 KB |   46632 KB |
|       from large pool |   94208 KB |  140288 KB |  140288 KB |   46080 KB |
|       from small pool |    1538 KB |    1538 KB |    2090 KB |     552 KB |
|---------------------------------------------------------------------------|
| Active memory         |   95746 KB |  141609 KB |  142378 KB |   46632 KB |
|       from large pool |   94208 KB |  140288 KB |  140288 KB |

In [7]:
audio = Audio(hp)
model = VoiceFilter(hp).cuda()
if hp.train.optimizer == 'adabound':
    optimizer = AdaBound(model.parameters(),
                         lr=hp.train.adabound.initial,
                         final_lr=hp.train.adabound.final)
elif hp.train.optimizer == 'adam':
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=hp.train.adam)
else:
    raise Exception("%s optimizer not supported" % hp.train.optimizer)

step = 0

print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  169500 KB |  234537 KB |  284458 KB |  114958 KB |
|       from large pool |  165817 KB |  230833 KB |  280197 KB |  114380 KB |
|       from small pool |    3683 KB |    3704 KB |    4261 KB |     578 KB |
|---------------------------------------------------------------------------|
| Active memory         |  169500 KB |  234537 KB |  284458 KB |  114958 KB |
|       from large pool |  165817 KB |  230833 KB |  280197 KB |

In [8]:
logger.info("Starting new training run")

criterion = nn.MSELoss()
_criterion = PowerLawCompLoss()

2021-11-24 17:00:18,393 - INFO - Starting new training run


In [9]:
model.train()

VoiceFilter(
  (conv): Sequential(
    (0): ZeroPad2d(padding=(3, 3, 0, 0), value=0.0)
    (1): Conv2d(1, 64, kernel_size=(1, 7), stride=(1, 1))
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): ZeroPad2d(padding=(0, 0, 3, 3), value=0.0)
    (5): Conv2d(64, 64, kernel_size=(7, 1), stride=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): ZeroPad2d(padding=(2, 2, 2, 2), value=0.0)
    (9): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): ZeroPad2d(padding=(2, 2, 4, 4), value=0.0)
    (13): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), dilation=(2, 1))
    (14): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): ReLU()
    (16): ZeroPad2d(padding=(2, 2, 8, 8), value=0.0)
    (17): Conv2d(64, 64, k

In [10]:
def get_gpu_allocated():
    print(torch.cuda.memory_allocated()/10**6, "MB")

get_gpu_allocated()

173.568 MB


In [11]:
import asyncio
import time

In [12]:
async def UploadToDrive(drive,save_path):
    print(f"started backup at{time.strftime('%X')}")
    drive.Upload(save_path, "1sWAUt5vfyD97Cq85J8_zuwMeX4tmfEiZ")
    print(f"finished backup at{time.strftime('%X')}")

In [13]:
accum = 0
accum_loss = 0

for dvec_mels, target_mag, mixed_mag in trainloader:
    target_mag = target_mag.cuda()
    mixed_mag = mixed_mag.cuda()
    
    print("After input to cuda: ", end="")
    get_gpu_allocated()

    dvec_list = list()
    for mel in dvec_mels:
        mel = mel.cuda()
        dvec = embedder(mel)
        print("\bAfter single dvec to cuda: ", end="")
        get_gpu_allocated()
        dvec_list.append(dvec)
    dvec = torch.stack(dvec_list, dim=0)
    dvec = dvec.detach()
    
    print("After dvec to cuda: ", end="")
    get_gpu_allocated()

    mask = model(mixed_mag, dvec)
    output = mixed_mag * mask
    
    print("After forward to cuda: ", end="")
    get_gpu_allocated()

    # output = torch.pow(torch.clamp(output, min=0.0), hp.audio.power)
    # target_mag = torch.pow(torch.clamp(target_mag, min=0.0), hp.audio.power)
    loss = criterion(output, target_mag)

    loss.backward()
    accum_loss += loss.item()
    accum += 1
    
    if accum % hp["train"]["grad_accumulate"] == 0:
        optimizer.step()
        optimizer.zero_grad()
        accum = 0
        step += 1
        accum_loss /= hp["train"]["grad_accumulate"]
        
        if accum_loss > 1e8 or math.isnan(accum_loss):
            logger.error("Loss exploded to %.02f at step %d!" % (accum_loss, step))
            raise Exception("Loss exploded")

        if step == 2100:
            break

        # write loss to tensorboard
        if step % hp.train.summary_interval == 0:
            writer.log_training(accum_loss, step)
            logger.info("Wrote summary at step %d" % step)

        accum_loss = 0
            
        # 1. save checkpoint file to resume training
        # 2. evaluate and save sample to tensorboard
        # backup brrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
        if step % hp.train.checkpoint_interval == 0:
            save_path = os.path.join(pt_dir, 'chkpt_%d.pt' % step)
            torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'step': step,
                'hp_str': hp_str,
            }, save_path)
            logger.info("Saved checkpoint to: %s" % save_path)
            validate(audio, model, embedder, testloader, writer, step)

            # drive.Upload(save_path, "1sWAUt5vfyD97Cq85J8_zuwMeX4tmfEiZ")
            asyncio.run(UploadToDrive(drive, save_path))

            # Nén file
            os.system(f'zip -j ./tensorboard.zip ./{log_dir}/*')
            drive.Upload('tensorboard.zip', "1sWAUt5vfyD97Cq85J8_zuwMeX4tmfEiZ")

After input to cuda: 179.357696 MB
After single dvec to cuda: 189.145088 MB
After single dvec to cuda: 197.579264 MB
After single dvec to cuda: 207.099904 MB
After single dvec to cuda: 217.975808 MB
After dvec to cuda: 217.979904 MB
After forward to cuda: 4124.961792 MB
After input to cuda: 299.422208 MB
After single dvec to cuda: 271.133184 MB
After single dvec to cuda: 279.563776 MB
After single dvec to cuda: 288.269824 MB
After single dvec to cuda: 293.425664 MB
After dvec to cuda: 293.42976 MB
After forward to cuda: 4192.329728 MB


2021-11-24 17:00:19,788 - INFO - Wrote summary at step 1


After input to cuda: 445.507072 MB
After single dvec to cuda: 423.109632 MB
After single dvec to cuda: 430.996992 MB
After single dvec to cuda: 439.698432 MB
After single dvec to cuda: 448.40192 MB
After dvec to cuda: 448.406016 MB
After forward to cuda: 4347.930624 MB
After input to cuda: 448.406016 MB
After single dvec to cuda: 423.161344 MB
After single dvec to cuda: 434.369024 MB
After single dvec to cuda: 443.888128 MB
After single dvec to cuda: 454.222848 MB
After dvec to cuda: 454.226944 MB
After forward to cuda: 4355.277824 MB


2021-11-24 17:00:20,491 - INFO - Wrote summary at step 2


After input to cuda: 455.781888 MB
After single dvec to cuda: 424.689152 MB
After single dvec to cuda: 430.677504 MB
After single dvec to cuda: 439.926272 MB
After single dvec to cuda: 445.57568 MB
After dvec to cuda: 445.579776 MB
After forward to cuda: 4344.21504 MB
After input to cuda: 444.781056 MB
After single dvec to cuda: 423.94368 MB
After single dvec to cuda: 433.46176 MB
After single dvec to cuda: 442.976256 MB
After single dvec to cuda: 447.606784 MB
After dvec to cuda: 447.61088 MB
After forward to cuda: 4347.542016 MB


...

After input to cuda: 441.85344 MB
After single dvec to cuda: 421.792256 MB
After single dvec to cuda: 430.765056 MB
After single dvec to cuda: 441.099264 MB
After single dvec to cuda: 451.435008 MB
After dvec to cuda: 451.439104 MB
After forward to cuda: 4350.963712 MB
After input to cuda: 451.439104 MB
After single dvec to cuda: 423.942656 MB
After single dvec to cuda: 426.39872 MB
After single dvec to cuda: 434.287616 MB
After single dvec to cuda: 439.864832 MB
After dvec to cuda: 439.868928 MB
After forward to cuda: 4342.022656 MB


2021-11-24 17:12:07,683 - INFO - Wrote summary at step 999


After input to cuda: 439.112704 MB
After single dvec to cuda: 423.1808 MB
After single dvec to cuda: 424.660992 MB
After single dvec to cuda: 427.116544 MB
After single dvec to cuda: 435.277824 MB
After dvec to cuda: 435.28192 MB
After forward to cuda: 4338.387968 MB
After input to cuda: 436.620288 MB
After single dvec to cuda: 424.499712 MB
After single dvec to cuda: 434.286592 MB
After single dvec to cuda: 438.370816 MB
After single dvec to cuda: 448.157184 MB
After dvec to cuda: 448.16128 MB
After forward to cuda: 4347.088896 MB


2021-11-24 17:12:08,389 - INFO - Wrote summary at step 1000
2021-11-24 17:12:08,763 - INFO - Saved checkpoint to: ./chkpt/first_try/chkpt_1000.pt


AttributeError: module 'asyncio' has no attribute 'run'

In [15]:
asyncio.run

AttributeError: module 'asyncio' has no attribute 'run'